# Rastreo de insumos
## Para las 3 sucursales analizar la cantidad que se envio durante una semana de domingo a sabado, restando las ventas.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, date, time, timedelta
import calendar


### Descarga de datos y creacion de diccionario

In [7]:
Tdata = pd.ExcelFile('tepeyac.xlsx')
Pdata = pd.ExcelFile('palomar.xlsx')
Idata = pd.ExcelFile('inglaterra.xlsx')
Idata.parse(-1)
sheet_to_df_map = {}
sheet_to_df_map['T']=Tdata.parse(-1)
sheet_to_df_map['P']=Pdata.parse(-1)
sheet_to_df_map['I']=Idata.parse(-1)



In [187]:
def filtrar(lista:'En este caso la tabla con los productos vendidos de toda la semana como diccionario',
            empresa:'La sucursal en especifico en la cual se filtrara',
            posicion:'Indice para buscar',
            comparacion:'Producto que se buscara'):
    '''Esta funcion filtrara de la lista de productos vendidos de las sucursales, este sera un diccionario por lo cual buscara
    el producto que se le especifique'''
    if(comparacion in lista[empresa]['descripcion'][posicion]):
        return posicion
    else:
        return -1
    

### Filtracion de un producto
Esto funciona para cualquier producto, solo faltaria extender en el parametro de comparacion, un map para todos los productos que se desean buscar

In [250]:
size=range(np.size(sheet_to_df_map['I']['descripcion']))
result=list(map(lambda e:list(map(lambda n:filtrar(sheet_to_df_map,e,n,'BUCHE'),range(np.size(sheet_to_df_map[e]['descripcion'])))),['I','P','T']))
result=list(map(lambda n:np.array(result[n])>0,range(3)))
x=list(map(lambda e,n:sheet_to_df_map[e].iloc[result[n],:],['I','P','T'],range(3)))


In [362]:
#Igual que en la linea anterior pero agregando una columna de zeros para la multiplicacion
y=list(map(lambda e,n:sheet_to_df_map[e].iloc[result[n],:].join(pd.DataFrame(np.ones(np.size(result[n])))),['I','P','T'],range(3)))



# Calculo del consumo del insumo
Esto aun se esta pensando en una logica mas especifica segun el insumo que es cambiaria la funcion de consumo, se debe de pensar de manera mas general

In [313]:
def consumo(producto,insumo):
    if("TORTA" in producto or "SUSHITORTA"in producto):
        if("1/2 "+insumo in producto):
            return array.append(.06)
        else:
            return array.append(.12)
    else:
        if("1/2 "+insumo in producto):
            return array.append(.03)
        else:
            return array.append(.06)


In [334]:
array=[]
z=list(map(lambda p:consumo(p,'BUCHE'),sheet_to_df_map['I'].iloc[result[0],1]))
y[0][0]*array